# Détection de palmiers avec YOLOv8 + Export ONNX pour Deepness (QGIS)

Ce notebook permet de :
1. Monter Google Drive et accéder au dataset
2. Installer les dépendances nécessaires
3. **Diagnostiquer** le dataset (labels, tailles de bboxes, images)
4. Entraîner un modèle **YOLOv8s** optimisé sur des images de palmiers
5. Exporter le modèle au format ONNX
6. Ajouter les métadonnées Deepness (plugin QGIS) au fichier ONNX
7. Sauvegarder le modèle final dans Google Drive

**Prérequis** : le dataset doit être dans Google Drive avec la structure suivante :
```
dataset_palmiers/
├── images/
│   ├── train/
│   └── val/
├── labels/
│   ├── train/
│   └── val/
└── palms.yaml
```

## 1. Monter Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2. Installer les dépendances

On installe `ultralytics` (YOLOv8) et `onnx` (pour manipuler les métadonnées du modèle exporté).

In [ ]:
!pip install ultralytics onnx --quiet

## 3. Définir le chemin du dataset et mettre à jour `palms.yaml`

On met à jour le fichier `palms.yaml` pour que le chemin (`path`) pointe vers le dossier dans Google Drive.

In [ ]:
import os
import yaml

# --- Trouver automatiquement le dataset dans Google Drive ---
DRIVE_ROOT = '/content/drive/MyDrive'

# Chemins possibles (ajoutez le vôtre si différent)
candidates = [
    os.path.join(DRIVE_ROOT, 'Recherche', 'dataset_palmiers'),
    os.path.join(DRIVE_ROOT, 'dataset_palmiers'),
    os.path.join(DRIVE_ROOT, 'recherche', 'dataset_palmiers'),
]

DATASET_DIR = None
for path in candidates:
    if os.path.isdir(path):
        DATASET_DIR = path
        break

# Si aucun candidat trouvé, afficher le contenu du Drive pour aider
if DATASET_DIR is None:
    print("❌ Dataset non trouvé dans les chemins suivants :")
    for p in candidates:
        print(f"   {p}")
    print(f"\n📂 Contenu de {DRIVE_ROOT} :")
    for item in sorted(os.listdir(DRIVE_ROOT)):
        full = os.path.join(DRIVE_ROOT, item)
        marker = '📁' if os.path.isdir(full) else '📄'
        print(f"   {marker} {item}")
    # Chercher récursivement un dossier dataset_palmiers
    print("\n🔍 Recherche de 'dataset_palmiers' dans le Drive...")
    for root, dirs, files in os.walk(DRIVE_ROOT):
        if 'dataset_palmiers' in dirs:
            found = os.path.join(root, 'dataset_palmiers')
            print(f"   ✅ Trouvé : {found}")
    raise FileNotFoundError("Modifiez DATASET_DIR avec le bon chemin ci-dessus.")

YAML_PATH = os.path.join(DATASET_DIR, 'palms.yaml')
assert os.path.isfile(YAML_PATH), f"Le fichier palms.yaml n'existe pas dans {DATASET_DIR}"

print(f"✅ Dataset trouvé : {DATASET_DIR}")

# Mettre à jour le path dans palms.yaml pour pointer vers Colab
with open(YAML_PATH, 'r') as f:
    config = yaml.safe_load(f)

config['path'] = DATASET_DIR

with open(YAML_PATH, 'w') as f:
    yaml.dump(config, f, default_flow_style=False, allow_unicode=True)

print("Configuration du dataset :")
print(yaml.dump(config, default_flow_style=False, allow_unicode=True))

# Vérifier la présence des images et labels
for split in ['train', 'val']:
    img_dir = os.path.join(DATASET_DIR, 'images', split)
    lbl_dir = os.path.join(DATASET_DIR, 'labels', split)
    n_img = len([f for f in os.listdir(img_dir) if f.endswith(('.tif', '.tiff', '.png', '.jpg'))])
    n_lbl = len([f for f in os.listdir(lbl_dir) if f.endswith('.txt')])
    print(f"  {split}: {n_img} images, {n_lbl} labels")

## 4. Diagnostic du dataset

Avant d'entraîner, on vérifie la qualité du dataset :
- Labels vides ou manquants
- Distribution des tailles de bounding boxes
- Nombre d'objets par image

Cela permet de détecter des problèmes en amont (annotations incorrectes, objets trop petits, etc.).

In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

print("=" * 60)
print("DIAGNOSTIC DU DATASET")
print("=" * 60)

all_widths, all_heights, objects_per_image = [], [], []
empty_labels, missing_labels = [], []

for split in ['train', 'val']:
    img_dir = os.path.join(DATASET_DIR, 'images', split)
    lbl_dir = os.path.join(DATASET_DIR, 'labels', split)

    images = sorted(glob.glob(os.path.join(img_dir, '*.*')))
    print(f"\n--- {split.upper()} ---")
    print(f"  Images trouvées : {len(images)}")

    for img_path in images:
        base = os.path.splitext(os.path.basename(img_path))[0]
        lbl_path = os.path.join(lbl_dir, base + '.txt')

        if not os.path.exists(lbl_path):
            missing_labels.append(f"{split}/{base}")
            objects_per_image.append(0)
            continue

        with open(lbl_path, 'r') as f:
            lines = [l.strip() for l in f.readlines() if l.strip()]

        if len(lines) == 0:
            empty_labels.append(f"{split}/{base}")
            objects_per_image.append(0)
            continue

        objects_per_image.append(len(lines))
        for line in lines:
            parts = line.split()
            if len(parts) >= 5:
                w, h = float(parts[3]), float(parts[4])
                all_widths.append(w)
                all_heights.append(h)

if missing_labels:
    print(f"\n⚠️  Labels MANQUANTS ({len(missing_labels)}) : {missing_labels[:10]}")
if empty_labels:
    print(f"⚠️  Labels VIDES ({len(empty_labels)}) : {empty_labels[:10]}")
if not missing_labels and not empty_labels:
    print("\n✅ Tous les labels sont présents et non-vides.")

total_objects = sum(objects_per_image)
print(f"\n📊 Statistiques :")
print(f"  Total d'objets annotés : {total_objects}")
print(f"  Objets/image — min: {min(objects_per_image)}, max: {max(objects_per_image)}, "
      f"moyenne: {np.mean(objects_per_image):.1f}, médiane: {np.median(objects_per_image):.0f}")

# Graphiques
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# Distribution du nombre d'objets par image
axes[0].hist(objects_per_image, bins=30, color='steelblue', edgecolor='white')
axes[0].set_title("Objets par image")
axes[0].set_xlabel("Nombre d'objets")
axes[0].set_ylabel("Nombre d'images")

# Distribution des largeurs de bbox (normalisées)
axes[1].hist(all_widths, bins=50, color='coral', edgecolor='white')
axes[1].set_title("Largeur des bboxes (normalisée)")
axes[1].set_xlabel("Largeur")
axes[1].axvline(np.median(all_widths), color='red', linestyle='--', label=f'médiane={np.median(all_widths):.3f}')
axes[1].legend()

# Distribution des hauteurs de bbox (normalisées)
axes[2].hist(all_heights, bins=50, color='mediumseagreen', edgecolor='white')
axes[2].set_title("Hauteur des bboxes (normalisée)")
axes[2].set_xlabel("Hauteur")
axes[2].axvline(np.median(all_heights), color='red', linestyle='--', label=f'médiane={np.median(all_heights):.3f}')
axes[2].legend()

plt.tight_layout()
plt.show()

# Alerte si bboxes très petites
small_threshold = 0.02  # 2% de l'image
n_small = sum(1 for w, h in zip(all_widths, all_heights) if w < small_threshold or h < small_threshold)
if n_small > 0:
    pct = n_small / len(all_widths) * 100
    print(f"\n⚠️  {n_small} bboxes ({pct:.1f}%) sont très petites (<{small_threshold*100}% de l'image).")
    print(f"   → La taille d'entrée imgsz=1024 aidera à les détecter.")
else:
    print(f"\n✅ Toutes les bboxes ont une taille raisonnable.")

## 5. Corriger les labels (BOM + classes incorrectes)

Deux problèmes courants détectés dans les labels :
1. **BOM UTF-8** (`\ufeff`) : caractère invisible en début de fichier, ajouté par certains éditeurs Windows
2. **Classes incorrectes** : certains fichiers utilisent la classe `17` au lieu de `0` — on remap tout vers la classe `0` (palmier unique)

In [ ]:
import glob

bom_fixed = 0
class_fixed = 0

for split in ['train', 'val']:
    lbl_dir = os.path.join(DATASET_DIR, 'labels', split)
    for lbl_file in sorted(glob.glob(os.path.join(lbl_dir, '*.txt'))):
        # Lire en binaire pour détecter le BOM
        with open(lbl_file, 'rb') as f:
            raw = f.read()

        # Supprimer le BOM UTF-8 (EF BB BF)
        had_bom = raw.startswith(b'\xef\xbb\xbf')
        if had_bom:
            raw = raw[3:]
            bom_fixed += 1

        # Décoder et corriger les classes
        text = raw.decode('utf-8')
        new_lines = []
        file_class_fixed = False
        for line in text.strip().split('\n'):
            if not line.strip():
                continue
            parts = line.strip().split()
            if len(parts) >= 5 and parts[0] != '0':
                parts[0] = '0'  # Remap toute classe vers 0 (palmier)
                file_class_fixed = True
            new_lines.append(' '.join(parts))

        if file_class_fixed:
            class_fixed += 1

        # Réécrire si modifié
        if had_bom or file_class_fixed:
            with open(lbl_file, 'w', encoding='utf-8', newline='\n') as f:
                f.write('\n'.join(new_lines) + '\n')

# Supprimer les fichiers .cache corrompus
for cache_file in glob.glob(os.path.join(DATASET_DIR, 'labels', '*.cache')):
    os.remove(cache_file)
    print(f"  Cache supprimé : {cache_file}")

print(f"\n✅ BOM supprimé de {bom_fixed} fichiers.")
print(f"✅ Classe corrigée (→ 0) dans {class_fixed} fichiers.")
print("   Les labels sont maintenant prêts pour YOLO.")

## 6. Convertir les images RGBA → RGB

Les GeoTIFF ont 4 canaux (RGBA) mais YOLOv8 attend 3 canaux (RGB). On convertit toutes les images en supprimant le canal alpha, et on les sauvegarde en PNG (format standard pour YOLO).

In [ ]:
from PIL import Image
import glob

converted = 0
for split in ['train', 'val']:
    img_dir = os.path.join(DATASET_DIR, 'images', split)
    for img_path in sorted(glob.glob(os.path.join(img_dir, '*.tif'))):
        img = Image.open(img_path)
        # Convertir en RGB si l'image a un canal alpha (RGBA)
        if img.mode == 'RGBA':
            img = img.convert('RGB')
        elif img.mode != 'RGB':
            img = img.convert('RGB')

        # Sauvegarder en PNG (même nom, extension .png)
        png_path = os.path.splitext(img_path)[0] + '.png'
        img.save(png_path)
        converted += 1

    # Supprimer les anciens .tif pour éviter les doublons
    for tif_path in glob.glob(os.path.join(img_dir, '*.tif')):
        os.remove(tif_path)

# Supprimer les .cache pour forcer YOLO à rescanner les nouvelles images
for cache_file in glob.glob(os.path.join(DATASET_DIR, 'labels', '*.cache')):
    os.remove(cache_file)

print(f"✅ {converted} images converties de RGBA/TIF → RGB/PNG.")
print(f"   Les fichiers .tif originaux ont été supprimés.")

## 7. Entraîner le modèle YOLOv8n (adapté petit dataset)

Avec seulement **38 images d'entraînement**, il faut éviter l'overfitting. On utilise **YOLOv8n** (nano, 3M params) au lieu de YOLOv8s (11M) et on applique une stratégie de **fine-tuning** :

**Adaptations au petit dataset :**
- `yolov8n.pt` : modèle léger → moins de risque d'overfitting
- `imgsz=640` : proche du natif 680px (pas d'upscale inutile)
- `freeze=10` : geler le backbone pré-entraîné, ne fine-tuner que la tête de détection
- `lr0=0.001` : learning rate bas pour du fine-tuning
- `copy_paste=0.3` : copie-colle d'objets entre images (augmentation très efficace)
- `mixup=0.15` : mélange d'images pour régulariser
- `epochs=200` / `patience=30` : plus de temps avec LR bas

In [ ]:
from ultralytics import YOLO

# YOLOv8n (nano) — mieux adapté à un petit dataset (38 images)
model = YOLO('yolov8n.pt')

# Entraînement optimisé pour petit dataset + vue aérienne
results = model.train(
    data=YAML_PATH,
    epochs=200,
    imgsz=640,           # proche du natif 680px, pas d'upscale
    batch=8,
    project=os.path.join(DATASET_DIR, 'runs'),
    name='palmier_detect',
    exist_ok=True,
    # --- Fine-tuning ---
    freeze=10,           # geler le backbone (couches 0-9), fine-tuner la tête
    lr0=0.001,           # LR bas pour fine-tuning
    lrf=0.01,            # LR final = lr0 * lrf
    cos_lr=True,         # cosine annealing
    # --- Early stopping ---
    patience=30,         # 30 époques sans amélioration → arrêt
    # --- Augmentations vue aérienne ---
    degrees=90.0,        # rotation ±90° (palmiers vus du dessus)
    flipud=0.5,          # retournement vertical (vue zénithale)
    fliplr=0.5,          # retournement horizontal
    scale=0.3,           # multi-scale modéré (petit dataset)
    mosaic=1.0,          # mosaic activé
    close_mosaic=20,     # désactiver mosaic les 20 dernières époques
    copy_paste=0.3,      # copie-colle d'objets entre images
    mixup=0.15,          # mélange d'images pour régulariser
    # --- Sauvegarde ---
    save=True,
    plots=True
)

## 6. Visualiser les résultats d'entraînement

Afficher les courbes de loss et les métriques de validation.

In [ ]:
from IPython.display import Image, display

run_dir = os.path.join(DATASET_DIR, 'runs', 'palmier_detect')

# Afficher les courbes de résultats
results_img = os.path.join(run_dir, 'results.png')
if os.path.exists(results_img):
    display(Image(filename=results_img, width=900))
else:
    print("Fichier results.png non trouvé.")

# Afficher la matrice de confusion
conf_img = os.path.join(run_dir, 'confusion_matrix.png')
if os.path.exists(conf_img):
    display(Image(filename=conf_img, width=600))

# Afficher des prédictions sur le set de validation
val_img = os.path.join(run_dir, 'val_batch0_pred.png')
if os.path.exists(val_img):
    display(Image(filename=val_img, width=900))

## 7. Évaluation détaillée sur le set de validation

On lance une évaluation formelle avec le meilleur modèle pour obtenir les métriques précises (mAP50, mAP50-95, précision, rappel).

In [ ]:
# Charger le meilleur modèle et lancer la validation
run_dir = os.path.join(DATASET_DIR, 'runs', 'palmier_detect')
best_model_path = os.path.join(run_dir, 'weights', 'best.pt')

val_model = YOLO(best_model_path)
metrics = val_model.val(data=YAML_PATH, imgsz=640, split='val')

print("=" * 50)
print("MÉTRIQUES DE VALIDATION")
print("=" * 50)
print(f"  Précision (P)   : {metrics.box.mp:.4f}")
print(f"  Rappel (R)      : {metrics.box.mr:.4f}")
print(f"  mAP@50          : {metrics.box.map50:.4f}")
print(f"  mAP@50-95       : {metrics.box.map:.4f}")
print()

# Interprétation
map50 = metrics.box.map50
if map50 >= 0.85:
    print("✅ Excellent ! Le modèle détecte très bien les palmiers.")
elif map50 >= 0.70:
    print("👍 Bon résultat. Peut être amélioré avec plus de données ou plus d'époques.")
elif map50 >= 0.50:
    print("⚠️  Résultat moyen. Vérifiez la qualité des annotations et augmentez les données.")
else:
    print("❌ Résultat faible. Le dataset ou les annotations nécessitent une révision.")

## 8. Exporter le meilleur modèle en ONNX

On charge le meilleur poids (`best.pt`) et on l'exporte au format ONNX avec `imgsz=640` (même taille que l'entraînement).

In [ ]:
# Charger le meilleur modèle entraîné
best_model_path = os.path.join(run_dir, 'weights', 'best.pt')
assert os.path.isfile(best_model_path), f"Modèle introuvable : {best_model_path}"

best_model = YOLO(best_model_path)

# Exporter en ONNX (opset=17 pour compatibilité avec Deepness/QGIS)
onnx_path = best_model.export(format='onnx', imgsz=640, opset=17)
print(f"Modèle ONNX exporté : {onnx_path}")

## 9. Ajouter les métadonnées Deepness au fichier ONNX

Le plugin **Deepness** pour QGIS attend des métadonnées spécifiques dans le modèle ONNX.
On les ajoute ici directement dans le fichier.

**Important** : ajustez la valeur `resolution` (en cm/pixel) selon la résolution réelle de vos orthophotos.

In [ ]:
import onnx
import json

# Charger le modèle ONNX
onnx_model = onnx.load(onnx_path)

# --- Supprimer toutes les métadonnées existantes (ajoutées par Ultralytics) ---
# Deepness fait json.loads() sur CHAQUE valeur → il faut que tout soit du JSON valide.
# Les métadonnées Ultralytics ne sont pas au bon format → on les supprime.
while len(onnx_model.metadata_props) > 0:
    onnx_model.metadata_props.pop()

# --- Métadonnées Deepness ---
# IMPORTANT : toutes les valeurs doivent être encodées avec json.dumps()
# car Deepness appelle json.loads() sur chaque valeur lue.
# Réf: https://github.com/PUTvision/qgis-plugin-deepness/blob/devel/tutorials/detection/cars_yolov7/car_detection__prepare_and_train.ipynb

class_names = {0: 'palmier'}  # clé int → json.dumps convertira en "0"

m1 = onnx_model.metadata_props.add()
m1.key = 'model_type'
m1.value = json.dumps('Detector')         # → '"Detector"'

m2 = onnx_model.metadata_props.add()
m2.key = 'class_names'
m2.value = json.dumps(class_names)         # → '{"0": "palmier"}'

m3 = onnx_model.metadata_props.add()
m3.key = 'resolution'
m3.value = json.dumps(30)                  # cm/pixel — à adapter !

m4 = onnx_model.metadata_props.add()
m4.key = 'det_conf'
m4.value = json.dumps(0.3)

m5 = onnx_model.metadata_props.add()
m5.key = 'det_iou_thresh'
m5.value = json.dumps(0.5)

m6 = onnx_model.metadata_props.add()
m6.key = 'det_type'
m6.value = json.dumps('YOLO_Ultralytics')  # → '"YOLO_Ultralytics"'

# Sauvegarder le modèle ONNX avec les métadonnées
output_onnx_path = os.path.join(DATASET_DIR, 'palmier_yolov8n_deepness.onnx')
onnx.save(onnx_model, output_onnx_path)

print(f"Modèle ONNX final sauvegardé : {output_onnx_path}")
print(f"Taille : {os.path.getsize(output_onnx_path) / 1024 / 1024:.1f} Mo")
print()
print("Métadonnées Deepness ajoutées :")
for prop in onnx_model.metadata_props:
    print(f"  {prop.key}: {prop.value}")
    # Vérification : chaque valeur doit être parsable par json.loads
    json.loads(prop.value)
print("\n✅ Toutes les valeurs sont du JSON valide.")

## 10. Vérifier les métadonnées du modèle ONNX

On relit le fichier ONNX pour confirmer que les métadonnées sont bien enregistrées.

In [ ]:
# Vérification
check_model = onnx.load(output_onnx_path)

print("=" * 50)
print("Métadonnées du modèle ONNX exporté")
print("=" * 50)
for prop in check_model.metadata_props:
    print(f"  {prop.key}: {prop.value}")
print()
print(f"Entrées du modèle :")
for inp in check_model.graph.input:
    shape = [d.dim_value if d.dim_value else d.dim_param for d in inp.type.tensor_type.shape.dim]
    print(f"  {inp.name}: {shape}")
print(f"Sorties du modèle :")
for out in check_model.graph.output:
    shape = [d.dim_value if d.dim_value else d.dim_param for d in out.type.tensor_type.shape.dim]
    print(f"  {out.name}: {shape}")

## 11. Télécharger le modèle (optionnel)

Le modèle est déjà sauvegardé dans Google Drive à l'emplacement :

```
Google Drive/Recherche/dataset_palmiers/palmier_yolov8n_deepness.onnx
```

Vous pouvez aussi le télécharger directement depuis Colab :

In [ ]:
from google.colab import files

# Décommenter la ligne suivante pour télécharger le modèle
# files.download(output_onnx_path)

## Utilisation dans QGIS avec Deepness

1. Ouvrez QGIS et installez le plugin **Deepness** depuis le gestionnaire d'extensions
2. Chargez votre orthophoto dans QGIS
3. Lancez Deepness : **Plugins > Deepness > Detection**
4. Sélectionnez le fichier `palmier_yolov8n_deepness.onnx`
5. Les paramètres (confiance, IoU, résolution) seront automatiquement lus depuis les métadonnées
6. Lancez l'inférence — les palmiers détectés apparaîtront comme une couche vectorielle